In [2]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [3]:
%%sql
drop table if exists ArrayTbl2;
CREATE TABLE ArrayTbl2
 (key   CHAR(1) NOT NULL,
    i   INTEGER NOT NULL,
  val   INTEGER,
  PRIMARY KEY (key, i));

/* AはオールNULL、Bは一つだけ非NULL、Cはオール非NULL */
INSERT INTO ArrayTbl2 VALUES('A', 1, NULL);
INSERT INTO ArrayTbl2 VALUES('A', 2, NULL);
INSERT INTO ArrayTbl2 VALUES('A', 3, NULL);
INSERT INTO ArrayTbl2 VALUES('A', 4, NULL);
INSERT INTO ArrayTbl2 VALUES('A', 5, NULL);
INSERT INTO ArrayTbl2 VALUES('A', 6, NULL);
INSERT INTO ArrayTbl2 VALUES('A', 7, NULL);
INSERT INTO ArrayTbl2 VALUES('A', 8, NULL);
INSERT INTO ArrayTbl2 VALUES('A', 9, NULL);
INSERT INTO ArrayTbl2 VALUES('A',10, NULL);
INSERT INTO ArrayTbl2 VALUES('B', 1, 3);
INSERT INTO ArrayTbl2 VALUES('B', 2, NULL);
INSERT INTO ArrayTbl2 VALUES('B', 3, NULL);
INSERT INTO ArrayTbl2 VALUES('B', 4, NULL);
INSERT INTO ArrayTbl2 VALUES('B', 5, NULL);
INSERT INTO ArrayTbl2 VALUES('B', 6, NULL);
INSERT INTO ArrayTbl2 VALUES('B', 7, NULL);
INSERT INTO ArrayTbl2 VALUES('B', 8, NULL);
INSERT INTO ArrayTbl2 VALUES('B', 9, NULL);
INSERT INTO ArrayTbl2 VALUES('B',10, NULL);
INSERT INTO ArrayTbl2 VALUES('C', 1, 1);
INSERT INTO ArrayTbl2 VALUES('C', 2, 1);
INSERT INTO ArrayTbl2 VALUES('C', 3, 1);
INSERT INTO ArrayTbl2 VALUES('C', 4, 1);
INSERT INTO ArrayTbl2 VALUES('C', 5, 1);
INSERT INTO ArrayTbl2 VALUES('C', 6, 1);
INSERT INTO ArrayTbl2 VALUES('C', 7, 1);
INSERT INTO ArrayTbl2 VALUES('C', 8, 1);
INSERT INTO ArrayTbl2 VALUES('C', 9, 1);
INSERT INTO ArrayTbl2 VALUES('C',10, 1);


*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

## やりたいこと
+ テーブルから「オール1」のエンティティだけを選択したい

In [3]:
%%sql
select distinct key
from ArrayTbl2 as a1
where not exists (
    select *
    from ArrayTbl2 as a2
    where a1.key = a2.key and
    -- valがnullもありうることに注意
    (a2.val is null or a2.val <> 1)
)

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


key
C


In [4]:
%%sql
select distinct key
from ArrayTbl2 as a1
where not exists (
    select *
    from ArrayTbl2 as a2
    where a1.key = a2.key and
    -- valがnullである対策としてcoalesceをつかうのもよい
    coalesce(a2.val, 0) <> 1
)

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


key
C


In [40]:
%%sql
-- havingを利用
select key
from ArrayTbl2
group by key
having max(val) = 1 and
    min(val) = 1

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


key
C


In [67]:
%%sql
-- select部で集合に対する条件を書く
select key
from (
    select key,
        case when max(val) = 1 and min(val) = 1 then 1 else 0 end as flag
    from ArrayTbl2
    group by key
) as tmp
where flag = 1

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


key
C


In [41]:
%%sql
-- havingを利用
select key
from ArrayTbl2
Group by key
having sum(case when val = 1 then 1 else 0 end) = 10

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


key
C


In [42]:
%%sql
-- allを利用(allの入力値は1次元のデータ)
select distinct key
from ArrayTbl2 as a1
where 1 = all(
    select val
    from ArrayTbl2 as a2
    where a1.key = a2.key
)

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


key
C
